<a href="https://colab.research.google.com/github/joaomerjam/ECON3916-33674-Statistical-Machine-Learning/blob/main/Lab9/Lab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

df = pd.read_csv('lalonde.csv')

Step 2: The Observational Failure

In [4]:
# naive comparison
naive_diff = df[df.treat==1]['re78'].mean() - df[df.treat==0]['re78'].mean()
print(f"Naive Difference in Means: ${naive_diff:,.2f}")

Naive Difference in Means: $-635.03


Step 3: Propensity Score Estimation

In [7]:
df.head()

,Unnamed: 0,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78
0,1,1,37,11,1,0,1,1,0.0,0.0,9930.0460
1,2,1,22,9,0,1,0,1,0.0,0.0,3595.8940
2,3,1,30,12,1,0,0,0,0.0,0.0,24909.4500
3,4,1,27,11,1,0,0,1,0.0,0.0,7506.1460
4,5,1,33,8,1,0,0,1,0.0,0.0,289.7899


In [9]:
# define covariates
x = df[['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're75','re78']]
y = df['treat']

# fit propensity model
logit = LogisticRegression(solver='liblinear')
logit.fit(x, y)

# generate scores
df['pscore'] = logit.predict_proba(x)[:,1]

In [10]:
df[['treat','pscore']].head()

,treat,pscore
0,1,0.417834
1,1,0.133574
2,1,0.762270
3,1,0.664419
4,1,0.669873


Step 4: The Matching Algorithm (Nearest Neighbor)

In [11]:
# separate groups
treated = df[df.treat==1]
control = df[df.treat==0]

# fit NN on control scores
nbrs = NearestNeighbors(n_neighbors=1).fit(control[['pscore']])

# find matches for treated scores
distances, indices = nbrs.kneighbors(treated[['pscore']])
matched_control = control.iloc[indices.flatten()]

# construct matched DataFrame
matched_df = pd.concat([treated, matched_control])

In [12]:
print(f"Original treated size: {len(treated)}")
print(f"Matched control size: {len(matched_control)}")

Original treated size: 185
Matched control size: 185


Step 5: Assessing Balance and Estimating the Effect

In [13]:
from scipy import stats

# T-test on raw data
diff = treated['re78'].mean() - control['re78'].mean()
t_stat, p_val = stats.ttest_ind(treated['re78'], control['re78'])

print(f"Raw Effect (Difference): ${diff:,.2f}")
print(f"P-value: {p_val:.4f}")


# isolate the matched outcomes
matched_treated = matched_df[matched_df.treat==1]['re78']
matched_control = matched_df[matched_df.treat==0]['re78']

# estimate the causal effect (T-test on matched data)
matched_diff = matched_treated.mean() - matched_control.mean()
t_stat, p_val = stats.ttest_ind(matched_treated, matched_control)


print(f"Recovered Effect (Matched Difference): ${matched_diff:,.2f}")
print(f"P-value: {p_val:.4f}")

Raw Effect (Difference): $-635.03
P-value: 0.3342
Recovered Effect (Matched Difference): $1,850.03
P-value: 0.0109
